In [28]:
import pandas as pd
import pandas as pd
import numpy as np
import os
import itertools
from AutomatedTraining import AutomatedTraining
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [29]:
# import data
df = pd.read_csv('data/merged_data.csv')

In [30]:
# Convert data types for memory efficiency
def convert_dtypes(df):
    df['order_value'] = df['order_value'].astype('float32')
    df['refund_value'] = df['refund_value'].astype('float32')
    df['num_items_ordered'] = df['num_items_ordered'].astype(float).round().astype('uint8')
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['first_order_datetime'] = pd.to_datetime(df['first_order_datetime'])
    df[['country_code', 'collect_type', 'payment_method']] = df[['country_code', 'collect_type', 'payment_method']].astype('category')
    return df

df = convert_dtypes(df)

In [31]:
# Payment method grouping
def group_payment_methods(payment_method):
    mapping = {
        'CreditCard': ['GenericCreditCard', 'CybersourceCreditCard', 'CybersourceApplePay', 'CreditCard'],
        'DigitalWallet': ['GCash', 'AFbKash', 'JazzCashWallet', 'AdyenBoost', 'PayPal'],
        'BankTransfer': ['XenditDirectDebit', 'RazerOnlineBanking'],
        'PaymentOnDelivery': ['Invoice', 'PayOnDelivery']
    }
    for key, values in mapping.items():
        if payment_method in values:
            return key
    return 'Others'

df['payment_method'] = df['payment_method'].map(group_payment_methods)

In [32]:
# Date transformations
def date_transformations(df):
    df['days_since_first_order'] = (df['order_date'] - df['first_order_datetime']).dt.days
    df = df.drop(columns=['first_order_datetime'])
    df['order_date_day_of_week'] = df['order_date'].dt.dayofweek
    df['order_date_day'] = df['order_date'].dt.day
    df['order_date_month'] = df['order_date'].dt.month
    df['order_date_year'] = df['order_date'].dt.year
    df = df.drop(columns=['order_date'])
    return df

df = date_transformations(df)
df = df.drop(columns=['order_id', 'customer_id'])

In [33]:
# Split data
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_cols = ['payment_method', 'country_code', 'collect_type']
numeric_columns = ['order_value', 'refund_value', 'num_items_ordered', 'days_since_first_order',
                   'order_date_day_of_week', 'order_date_day', 'order_date_month', 'order_date_year']

In [34]:
tracker = AutomatedTraining('Automated Training')
tracker.run_experiments(X_train, y_train, X_test, y_test, categorical_cols, numeric_columns)

Starting automated training run: RA_202502141145_automated


c:\Users\yongj\anaconda3\envs\PAI_env\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/02/14 11:45:46 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '      country_code  mobile_verified  num_orders_last_50days 

Completed automated training run: RA_202502141145_automated
Test F1 Score: 0.7595
🏃 View run RA_202502141145_automated at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/21/runs/e009ccfe79234dd7b32022438ecf6699
🧪 View experiment at: https://dagshub.com/REHXZ/PAI_CA2.mlflow/#/experiments/21
Ending automated training run


(Pipeline(steps=[('preprocessor',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('scaler', MinMaxScaler(),
                                                   ['order_value',
                                                    'refund_value',
                                                    'num_items_ordered',
                                                    'days_since_first_order',
                                                    'order_date_day_of_week',
                                                    'order_date_day',
                                                    'order_date_month',
                                                    'order_date_year']),
                                                  ('encoder',
                                                   OneHotEncoder(handle_unknown='ignore'),
                                                   ['payment_method',
                          